In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
from sklearn.kernel_ridge import KernelRidge

In [2]:
seg_list = os.listdir('../input/test/')

In [3]:
seg_list[0]

'seg_1a671a.csv'

In [6]:
seg_df = pd.read_csv('../input/test/{}'.format(seg_list[0]))

In [11]:
seg_df.head()

,acoustic_data
0,5
1,2
2,0
3,0
4,-1


In [9]:
seg_df.shape[0] // step

150

In [21]:
test_df = pd.DataFrame(columns=('seg_id', 'acoustic_data_mean'))
step = 1000
seg_id = 'seg_1a671a'
seg_id_lst = [seg_id for _ in range(seg_df.shape[0] // step)]
acoustic_data_mean_lst = [seg_df.iloc[i:i+step]['acoustic_data'].mean() for i in range(seg_df.shape[0] // step)]
'''
for i in range(seg_df.shape[0] // step):
    test_df['seg_id'] = seg_id
    test_df['acoustic_data_mean'] = seg_df.iloc[i:i+step]['acoustic_data'].mean()
'''

"\nfor i in range(seg_df.shape[0] // step):\n    test_df['seg_id'] = seg_id\n    test_df['acoustic_data_mean'] = seg_df.iloc[i:i+step]['acoustic_data'].mean()\n"

In [23]:
test_df['seg_id'] = seg_id_lst

In [24]:
test_df['acoustic_data_mean'] = acoustic_data_mean_lst

In [25]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
seg_id                150 non-null object
acoustic_data_mean    150 non-null float64
dtypes: float64(1), object(1)
memory usage: 2.4+ KB


In [26]:
test_df.head()

,seg_id,acoustic_data_mean
0,seg_1a671a,4.085
1,seg_1a671a,4.076
2,seg_1a671a,4.069
3,seg_1a671a,4.063
4,seg_1a671a,4.061


In [28]:
%%time
train_df = pd.read_csv('../input/train/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})

CPU times: user 2min 19s, sys: 7.64 s, total: 2min 27s
Wall time: 2min 27s


In [48]:
rows = 15000
segments = train_df.shape[0] // rows
#rows = 150_000
#rows = 150_00
#segments = int(np.floor(train_df.shape[0] / rows))

In [49]:
print(rows)
print(segments)

15000
41943


In [50]:
X_tr = pd.DataFrame(index=range(segments), dtype=np.float64)

y_tr = pd.DataFrame(index=range(segments), dtype=np.float64, columns=['time_to_failure'])

In [51]:
print(X_tr.shape)
print(y_tr.shape)

(41943, 0)
(41943, 1)


In [31]:
subsegments = 10

In [52]:
for segment in tqdm_notebook(range(segments)):
    seg = train_df.iloc[segment*rows:segment*rows+rows]
    #for subseg in range(subsegments):
    x = pd.Series(seg['acoustic_data'].values)
    y = 0
    #print(seg.shape)
    try:
        y = seg['time_to_failure'].values[-1]
    except:
        print("seg:\n", seg)
        print("prev_seg:\n", prev_seg)
        raise Exception("Stop!")
    prev_seg = seg
    y_tr.loc[segment, 'time_to_failure'] = y
    X_tr.loc[segment, 'mean'] = x.mean()
    #X_tr.loc[segment, 'std'] = x.std()
    #X_tr.loc[segment, 'max'] = x.max()
    #X_tr.loc[segment, 'min'] = x.min()

In [56]:
seg_chunk_len = 15000
chunks_num = 10
file_num = 20

In [57]:
X_test = pd.DataFrame(columns=['seg_id', 'mean', 'std'], dtype=np.float64, index=range(file_num * chunks_num))

In [59]:
test_idx = 0
for i, seg_filename in enumerate(tqdm_notebook(os.listdir('../input/test/')[:file_num])):
    seg_df = pd.read_csv('../input/test/{}'.format(seg_filename))
    seg_id = seg_filename[:-4]
    x = pd.Series(seg_df['acoustic_data'].values)
    for chunk_idx in range(chunks_num):
        x_chunk = x[chunk_idx * seg_chunk_len:(chunk_idx + 1) * seg_chunk_len]
        X_test.iloc[test_idx]['seg_id'] = seg_id
        X_test.iloc[test_idx]['mean'] = x_chunk.mean()
        X_test.iloc[test_idx]['std'] = x_chunk.std()
        #X_test.loc[seg_id, 'max'] = x_chunk.max()
        #X_test.loc[seg_id, 'min'] = x_chunk.min()
        test_idx += 1

In [60]:
X_test.shape

(200, 3)

In [61]:
X_test.head(20)

,seg_id,mean,std
0,NaN,3.933666666666667,4.735647964780742
1,NaN,3.997000000000000,4.465518304024918
2,NaN,4.210200000000000,3.516064107367850
3,NaN,4.081666666666667,4.129503692003642
4,NaN,4.003666666666667,5.735373820110826
5,NaN,3.991666666666667,4.538461971273220
6,NaN,4.056933333333333,4.673323006749827
7,NaN,3.933733333333334,6.712511644910651
8,NaN,3.832266666666666,8.034536023535924
9,NaN,3.806600000000000,17.325511286234008


In [62]:
X_test.tail(20)

,seg_id,mean,std
180,NaN,4.681400000000000,4.416030736060471
181,NaN,4.589733333333333,3.606422282678632
182,NaN,4.555800000000000,5.481492145733943
183,NaN,4.569933333333333,3.340357245563827
184,NaN,4.780866666666666,5.509162426382518
185,NaN,4.795333333333334,7.374677661674201
186,NaN,4.505666666666666,6.283592400672251
187,NaN,4.645800000000000,3.839200219301965
188,NaN,4.784866666666667,4.810155245774796
189,NaN,4.681866666666667,4.335148296149225
